For this week, I turned a composition from the Fonts assignment to be interactive based on Allison's [Interactive Widgets](https://github.com/aparrish/material-of-language/blob/master/interactive-widgets.ipynb) tutorial.


------
First, load all libraries and helper functions from week 9.

In [1]:
import ipywidgets as widgets
from flat import document, rgb, rgba, font, strike, shape
from flat.command import moveto, quadto, curveto, lineto, closepath
from bezmerizing import Polyline, Path
from numpy.random import uniform, normal, choice
from math import sqrt, floor
import numpy as np
from copy import copy

In [2]:
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

In [3]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]

In [4]:
f = font.open("./ZCOOLXiaoWei-Regular.ttf")

In [5]:
def nudge(cmd, x, y):
    c = copy(cmd)
    if type(cmd) in (moveto, lineto, quadto, curveto):
        c.x += x
        c.y += y
    if type(cmd) in (quadto, curveto):
        c.x1 += x
        c.y1 += y
    if type(cmd) == curveto:
        c.x2 += x
        c.y2 += y
    return c

In [6]:
def displace_each_closedpaths(ch, x_scale, y_scale):
    path_orig = glyphcommands(f, ch)
    path_cmds = []
    path_x_off = normal(0, x_scale)
    path_y_off = normal(0, y_scale)
    for cmd in path_orig:
        if hasattr(cmd, 'x') and hasattr(cmd, 'y'):
            path_cmds.append(nudge(cmd, x=path_x_off, y=path_y_off))
        else:
            path_x_off = normal(0, x_scale)
            path_y_off = normal(0, y_scale)
    path_mod = Path(path_cmds)
    return path_mod

In [7]:
def get_scale_by_center(cx, cy, center_x, center_y, max_dist, base_scale):
    dist = sqrt(pow(cx - center_x, 2) + pow(cy - center_y, 2))
    scale = np.interp(dist,[0,max_dist],[0,base_scale])
    return scale

In [8]:
def disp_circle(center_x=15, center_y=15):
    size_pt = 24

    width = 740 # 5in
    height = 660 # 4in
    border = 9 # 1/8in
    lineheight = size_pt # size of each line

    page = document(width, height, 'pt').addpage()
    pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 255))
    factor = size_pt / f.density

    xoff = border
    yoff = border

    cx = 0
    cy = 0

    center_x = center_x
    center_y = center_y

    for line in open("displace_ny.txt", encoding="utf-8"):
        line = line.strip() # remove trailing whitespace
        for ch in line:
            displace_scale = 1000 - get_scale_by_center(cx, cy * 2, center_x, center_y, 20, 1000)
            glyph_path = (displace_each_closedpaths(ch, displace_scale, displace_scale)
                      .scale(factor)
                      .translate(xoff, 60 + yoff))
            page.place(pen.path(glyph_path))
            xoff += advancefor(f, ch) * factor
            cx += 1
        xoff = border
        yoff += lineheight
        cx = 0
        cy += 1

    show(page)

In [9]:
disp_circle()

My goal is to make the above composition to be interactive. So I wrapped it in a function called disp_circle, and add int sliders to control the x and y position of the displacement circle.

In [10]:
widgets.interact(
    disp_circle,
    center_x=widgets.IntSlider(min=0, max=50, value=10),
    center_y=widgets.IntSlider(min=0, max=50, value=10))

interactive(children=(IntSlider(value=10, description='center_x', max=50), IntSlider(value=10, description='ce…

<function __main__.disp_circle(center_x=15, center_y=15)>

Then I added revised the displace function to support random seeding, and added control for radius, max dispace scale, seed value, and font size.

In [11]:
def displace_each_closedpaths_seed(ch, x_scale, y_scale, use_seed=True, seed=66):
    if (use_seed):
        np.random.seed(seed)
    else:
        np.random.seed(None)
    path_orig = glyphcommands(f, ch)
    path_cmds = []
    path_x_off = normal(0, x_scale)
    path_y_off = normal(0, y_scale)
    for cmd in path_orig:
        if hasattr(cmd, 'x') and hasattr(cmd, 'y'):
            path_cmds.append(nudge(cmd, x=path_x_off, y=path_y_off))
        else:
            path_x_off = normal(0, x_scale)
            path_y_off = normal(0, y_scale)
    path_mod = Path(path_cmds)
    return path_mod

In [12]:
def disp_circle_2(center_x=15, center_y=15, radius=20, max_displace_scale=500, use_seed=True, seed=66, size_pt=24):
    size_pt = size_pt

    width = 740 # 5in
    height = 660 # 4in
    border = 9 # 1/8in
    lineheight = size_pt # size of each line

    page = document(width, height, 'pt').addpage()
    pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 255))
    factor = size_pt / f.density

    xoff = border
    yoff = border * 3

    cx = 0
    cy = 0

    center_x = center_x
    center_y = center_y

    for line in open("displace_ny.txt", encoding="utf-8"):
        line = line.strip() # remove trailing whitespace
        for ch in line:
            displace_scale = max_displace_scale - get_scale_by_center(cx, cy * 2, center_x, center_y, radius, max_displace_scale)
            glyph_path = (displace_each_closedpaths_seed(ch, displace_scale, displace_scale, use_seed, seed)
                      .scale(factor)
                      .translate(xoff, yoff))
            page.place(pen.path(glyph_path))
            xoff += advancefor(f, ch) * factor
            cx += 1
        xoff = border
        yoff += lineheight
        cx = 0
        cy += 1

    show(page)

In [13]:
x_slider = widgets.IntSlider(
    description="center x", min=0, max=50, value=30,
    continuous_update=False)
y_slider = widgets.IntSlider(
    description="center y", min=0, max=50, value=20,
    continuous_update=False)
radius_slider = widgets.IntSlider(
    description="radius", min=0, max=40, value=20,
    continuous_update=False)
max_slider = widgets.IntSlider(
    description="max scale", min=0, max=1000, value=600, step=50,
    continuous_update=False)
size_slider = widgets.IntSlider(
    description="pt size", min=0, max=40, value=20,
    continuous_update=False)
seed_checkbox = widgets.Checkbox(
    description="Use random seed?",
    value=True
)
seed_input = widgets.BoundedIntText(
    description="seed value",
    min=0,
    max=99999,
    value=23333
)
layout = widgets.VBox([
    widgets.HBox([x_slider, y_slider]),
    widgets.HBox([radius_slider, max_slider]),
    widgets.HBox([seed_checkbox, seed_input]),
    widgets.HBox([size_slider]),
])
output = widgets.interactive_output(
    disp_circle_2,
    {
        'center_x': x_slider,
        'center_y': y_slider,
        'radius': radius_slider,
        'max_displace_scale': max_slider,
        'use_seed': seed_checkbox,
        'seed': seed_input,
        'size_pt': size_slider
        
    }
)
display(layout, output)

Output()

And finally, create an animation by moving the center x and y along a path created in week 4's assignment using ffmpeg.

In [14]:
def page2png(page, dpi=72):
    try:
        import cairosvg
        png_bytes = cairosvg.svg2png(page.svg(), dpi=dpi)
    except ModuleNotFoundError:
        png_bytes = page.image(ppi=dpi, kind='rgb').png()
    return png_bytes

In [15]:
import pandas
df = pandas.read_csv('mouse.csv', index_col='id')
print(df)

            x         y
id                     
0    0.048583  0.222222
1    0.048583  0.222222
2    0.048583  0.222222
3    0.048583  0.222222
4    0.048583  0.222222
..        ...       ...
513  0.779352  0.848325
514  0.779352  0.848325
515  0.779352  0.848325
516  0.779352  0.848325
517  0.779352  0.848325

[518 rows x 2 columns]


In [23]:
from itertools import chain

# canvas
width = 740 # 5in
height = 660 # 4in

def render(step=0):
    size_pt = 20

    border = 9 # 1/8in
    lineheight = size_pt # size of each line

    page = document(width, height, 'pt').addpage()
    background = shape().nostroke().fill(rgb(255, 255, 255))
    brush = shape().nostroke().fill(rgb(40, 40, 40))
    page.place(background.rectangle(0, 0, width, height))
    pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 255))
    factor = size_pt / f.density

    xoff = border
    yoff = border * 3

    cx = 0
    cy = 0

    center_x = floor(df.iloc[step, 0] * 50)
    center_y = floor(df.iloc[step, 1] * 50)

    for line in open("displace_ny.txt", encoding="utf-8"):
        line = line.strip() # remove trailing whitespace
        for ch in line:
            displace_scale = 600 - get_scale_by_center(cx, cy * 2, center_x, center_y, 20, 600)
            glyph_path = (displace_each_closedpaths_seed(ch, displace_scale, displace_scale, True, 23333)
                      .scale(factor)
                      .translate(xoff, yoff))
            page.place(pen.path(glyph_path))
            xoff += advancefor(f, ch) * factor
            cx += 1
        xoff = border
        yoff += lineheight
        cx = 0
        cy += 1

    return page

In [24]:
show(render(400))

In [20]:
!mkdir render

In [25]:
print(df.shape[0])
for i in range(df.shape[0]):
    print(i, end=" ")
    fname = f"render/image{i:05}.png"
    png_data = page2png(render(i))
    with open(fname, "wb") as fh:
        fh.write(png_data)

518
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 27

In [29]:
!ffmpeg -loglevel warning -y \
    -framerate 80 -f image2 -i render/image%05d.png \
    -vcodec libx264 -crf 10 -pix_fmt yuv420p render/ny.mp4

The animation video can be found here: [vimeo link](https://vimeo.com/410488566)